In [1]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from haversine import haversine

### Connect to RDS

In [2]:
endpoint = 'dao.cdqd2bwz9q4s.ap-northeast-2.rds.amazonaws.com'
schema_name = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema_name )
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

### SQL to Pandas DataFrame

In [38]:
table_name = "MainData_inner_현수"
sql = "SELECT * FROM {}".format(table_name)
df_maindata_inner = pd.read_sql(sql, db_connection)
df_maindata_inner = df_maindata_inner.iloc[:,:1]
df_maindata_inner

,row_names
0,1
1,2
2,3
3,4
4,5
...,...
515,535
516,536
517,537
518,538


### 유도울타리현황_2021

In [39]:
table_name = "유도울타리현황_2021"
sql = "SELECT *  FROM {}".format(table_name)
df_etc_fence = pd.read_sql(sql, db_connection)
df_etc_fence

,본부,지사,노선,설치시점,설치종점,방향,연장,높이,설치년도
0,수도권본부,경기광주,제2중부선,353,353,부산,30,2,2011
1,수도권본부,경기광주,제2중부선,353,353,부산,150,2,2011
2,수도권본부,경기광주,제2중부선,353,354,부산,130,2,2011
3,수도권본부,경기광주,제2중부선,354,354,부산,90,2,2011
4,수도권본부,경기광주,제2중부선,354,354,부산,70,2,2011
...,...,...,...,...,...,...,...,...,...
15419,충북본부,충주,중부내륙선,208,208,서울,175,1,2020
15420,충북본부,충주,중부내륙선,208,208,서울,10,2,2020
15421,충북본부,충주,중부내륙선,209,209,서울,135,2,2020
15422,충북본부,충주,중부내륙선,209,209,서울,20,1,2020


In [40]:
df_etc_fence_preprocessing = df_etc_fence.groupby(['노선', '설치시점']).agg({'설치종점':'max', '연장':'sum', '높이':'mean'})
df_etc_fence_preprocessing.reset_index(inplace=True)
df_etc_fence_preprocessing['시종점길이'] = (df_etc_fence_preprocessing['설치종점'] - df_etc_fence_preprocessing['설치시점'])
df_etc_fence_preprocessing['설치시점'] = df_etc_fence_preprocessing['설치시점'].astype('float')
df_etc_fence_preprocessing

,노선,설치시점,설치종점,연장,높이,시종점길이
0,경부선,3.0,3,270,2.000000,0
1,경부선,4.0,5,470,2.000000,1
2,경부선,5.0,6,492,2.000000,1
3,경부선,6.0,7,1506,1.636364,1
4,경부선,7.0,8,738,1.666667,1
...,...,...,...,...,...,...
2874,호남선의 지선,48.0,49,720,2.000000,1
2875,호남선의 지선,49.0,49,360,2.000000,0
2876,호남선의 지선,51.0,52,120,1.000000,1
2877,호남선의 지선,52.0,53,1095,1.800000,1


### 한국도로공사_도로중심선

In [41]:
file_name = "한국도로공사_도로중심선.csv"
df_etc_road = pd.read_csv(file_name, sep=",", header=0, encoding="cp949")
df_etc_road['이정'] = df_etc_road['이정'].astype('string')
df_etc_road.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48241 entries, 0 to 48240
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   노선번호       48241 non-null  int64  
 1   도로명        48241 non-null  object 
 2   이정         48241 non-null  string 
 3   X좌표값       48241 non-null  float64
 4   Y좌표값       48241 non-null  float64
 5   GRS80X좌표값  48241 non-null  float64
 6   GRS80Y좌표값  48241 non-null  float64
dtypes: float64(4), int64(1), object(1), string(1)
memory usage: 2.6+ MB


In [42]:
target_idx = []
for i in range(len(df_etc_road)):
    if df_etc_road.loc[i, '이정'][-1] != '0':
        target_idx.append(i)
        
df_etc_road.drop(target_idx, axis=0, inplace=True)

In [43]:
df_etc_road.reset_index(inplace=True, drop=True)
df_etc_road['이정'] = df_etc_road['이정'].astype('float')
df_etc_road

,노선번호,도로명,이정,X좌표값,Y좌표값,GRS80X좌표값,GRS80Y좌표값
0,10,경부선,0.0,35.246578,129.093207,390524.290167,296459.942628
1,10,경부선,1.0,35.253629,129.100005,391126.574702,297255.473039
2,10,경부선,2.0,35.261977,129.104050,391475.199578,298189.576852
3,10,경부선,3.0,35.270386,129.107884,391804.314426,299130.211426
4,10,경부선,4.0,35.278963,129.106610,391668.093399,300079.508662
...,...,...,...,...,...,...,...
4837,6000,부산외곽순환선,44.0,35.280043,129.165551,397029.027406,300314.946152
4838,6000,부산외곽순환선,45.0,35.278286,129.176331,398014.261102,300141.393525
4839,6000,부산외곽순환선,46.0,35.279118,129.187196,399000.981241,300255.474773
4840,6000,부산외곽순환선,47.0,35.280547,129.198049,399985.062982,300435.904279


### 도로명, 기점 거리를 기준으로 Join

In [44]:
li_latitude = [None] * len(df_etc_fence_preprocessing)
li_longitude = [None] * len(df_etc_fence_preprocessing)
li_idx = []
for idx in range(len(df_etc_fence_preprocessing)):
    li_idx.append(idx)
    for i in range(len(df_etc_road)):
        # 도로명과 노선명이 일치하는지 확인
        if df_etc_road.iloc[i,1] == df_etc_fence_preprocessing.iloc[idx,0]:
            # 이정과 설치시점이 같은지 확인
            if df_etc_road.iloc[i,2] == df_etc_fence_preprocessing.iloc[idx,1]:
                # 좌표값 저장
                li_latitude[idx] = df_etc_road.iloc[i,3]
                li_longitude[idx] = df_etc_road.iloc[i,4]
                break
print(len(li_latitude))
print(len(li_longitude))

2879
2879


In [45]:
if len(li_latitude) == len(li_longitude):
    df_etc_fence_preprocessing['위도'] = li_latitude
    df_etc_fence_preprocessing['경도'] = li_longitude
    print(True)
else:
    print(False)
df_etc_fence_preprocessing

True


,노선,설치시점,설치종점,연장,높이,시종점길이,위도,경도
0,경부선,3.0,3,270,2.000000,0,35.270386,129.107884
1,경부선,4.0,5,470,2.000000,1,35.278963,129.106610
2,경부선,5.0,6,492,2.000000,1,35.286647,129.101121
3,경부선,6.0,7,1506,1.636364,1,35.291514,129.091887
4,경부선,7.0,8,738,1.666667,1,35.297901,129.084194
...,...,...,...,...,...,...,...,...
2874,호남선의 지선,48.0,49,720,2.000000,1,36.400681,127.359373
2875,호남선의 지선,49.0,49,360,2.000000,0,36.406366,127.367995
2876,호남선의 지선,51.0,52,120,1.000000,1,36.412060,127.388434
2877,호남선의 지선,52.0,53,1095,1.800000,1,36.409214,127.399012


In [46]:
df_etc_fence_preprocessing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879 entries, 0 to 2878
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   노선      2879 non-null   object 
 1   설치시점    2879 non-null   float64
 2   설치종점    2879 non-null   int64  
 3   연장      2879 non-null   int64  
 4   높이      2879 non-null   float64
 5   시종점길이   2879 non-null   int64  
 6   위도      2854 non-null   float64
 7   경도      2854 non-null   float64
dtypes: float64(4), int64(3), object(1)
memory usage: 180.1+ KB


### 결측치 처리

In [47]:
df_etc_fence_preprocessing[df_etc_fence_preprocessing['위도'].isnull()]

,노선,설치시점,설치종점,연장,높이,시종점길이,위도,경도
404,광주대구선,178.0,179,640,2.000000,1,NaN,NaN
405,광주대구선,181.0,181,270,2.000000,0,NaN,NaN
628,남해제2지선,146.0,147,1436,2.000000,1,NaN,NaN
629,남해제2지선,147.0,147,276,2.000000,0,NaN,NaN
851,당진대전선,92.0,92,960,1.000000,0,NaN,NaN
994,동해선,123.0,123,948,1.277778,0,NaN,NaN
996,동해선,125.0,125,855,1.250000,0,NaN,NaN
997,동해선,151.0,151,794,1.555556,0,NaN,NaN
1005,밀양울산선,99.0,100,185,1.000000,1,NaN,NaN
1032,밀양울산선,145.0,146,2381,1.482759,1,NaN,NaN


In [48]:
df_etc = df_etc_fence_preprocessing[['위도','경도','노선','연장','높이']]
df_etc

,위도,경도,노선,연장,높이
0,35.270386,129.107884,경부선,270,2.000000
1,35.278963,129.106610,경부선,470,2.000000
2,35.286647,129.101121,경부선,492,2.000000
3,35.291514,129.091887,경부선,1506,1.636364
4,35.297901,129.084194,경부선,738,1.666667
...,...,...,...,...,...
2874,36.400681,127.359373,호남선의 지선,720,2.000000
2875,36.406366,127.367995,호남선의 지선,360,2.000000
2876,36.412060,127.388434,호남선의 지선,120,1.000000
2877,36.409214,127.399012,호남선의 지선,1095,1.800000


### 좌표간 거리 계산

In [64]:
table_name = "MainData"
sql = "SELECT * FROM {}".format(table_name)
df_maindata_inner_origin = pd.read_sql(sql, db_connection)
df_maindata_inner_origin.rename(columns={'지형기호(5km)':'지형기호(2.3km)'} ,inplace=True)
df_maindata_inner_origin

,번호,위도,경도,생태통로_유형,차선_수,등산객_이용빈도,생태통로_효율성,이용확인종,이용확인종_수,폭(m),...,산책로까지의 최단 거리(km),농가까지의 거리(km),농가까지의 거리(m),도로 최고제한속도(km/h),주변 동물종 개수,주변동물 출현빈도,지형기호(2.3km),하천거리(km),식물군락명,등산로까지 최단거리(km)
0,1495,35.36019,128.06375,0,1,1,3,None,0,NaN,...,32.993820,0.223706,223.706303,80,2,2,X,134.468063,소나무군락,3.157026
1,1277,37.03408,128.40661,1,0,0,3,None,0,NaN,...,189.603501,0.303690,303.690012,60,3,5,X,434.781679,소나무-신갈나무군락,1.087578
2,1278,36.89832,128.30563,1,0,0,1,고라니 멧돼지 파충류,3,NaN,...,172.236247,0.011154,11.153787,60,0,0,X,74.780224,소나무군락,3.070682
3,1279,36.89687,128.30581,1,0,0,1,고라니 멧돼지 파충류,3,NaN,...,172.092457,0.012767,12.767402,60,0,0,X,125.168772,소나무군락,2.921190
4,1090,36.71336,127.42200,0,2,5,1,개 고양이 다람쥐,3,8.23,...,140.219401,0.183505,183.504631,80,0,0,X,2275.200182,아까시나무-리기다소나무식재림,0.025083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,1089,36.71278,127.42056,0,2,0,0,고라니 멧돼지,2,13.30,...,140.166888,0.287976,287.976255,80,0,0,X,2274.279975,아까시나무-리기다소나무식재림,0.025617
493,1087,36.71722,127.41778,0,1,5,0,고라니 멧돼지,2,13.60,...,140.681388,0.119958,119.958413,80,0,0,X,1722.566263,아까시나무식재림,0.021407
494,1088,36.71222,127.42528,0,1,5,0,고라니 멧돼지,2,13.30,...,140.066912,0.116240,116.239892,80,0,0,X,2532.215687,아까시나무-리기다소나무식재림,0.008204
495,1091,36.76750,127.47333,0,0,0,0,멧돼지 고라니,2,7.00,...,145.890079,0.023978,23.977596,60,0,0,X,2096.234333,상수리나무군락,0.878327


In [65]:
df_etc_latitude = df_etc["위도"]
df_etc_longitude = df_etc["경도"]
df_etc_length = df_etc["연장"]
df_etc_height = df_etc["높이"]
df_etc_longitude = df_etc_longitude.values.tolist()
df_etc_latitude = df_etc_latitude.values.tolist()
df_etc_length = df_etc_length.values.tolist()
df_etc_height = df_etc_height.values.tolist()

df_latitude = df_maindata_inner_origin["위도"]
df_longitude = df_maindata_inner_origin["경도"]
df_main_longitude = df_longitude.values.tolist()
df_main_latitude = df_latitude.values.tolist()

In [66]:
print(len(df_etc_latitude))
print(len(df_etc_longitude))
print(len(df_latitude))
print(len(df_longitude))

2879
2879
497
497


In [67]:
def is_fence(n, fence_info_return = False):
    # 반경 nkm 이내의 가장 가까운 유도울타리

    fence_lat = [0] * len(df_latitude)
    fence_lng = [0] * len(df_longitude)
    fence_length = [0] * len(df_latitude)
    fence_height = [0] * len(df_latitude)

    cnt = 0
    li_idx = []
    li_distance = []

    for idx in range(len(df_latitude)):
        distance_min = 500
        distance_min_idx = 0
        start = (df_latitude[idx], df_longitude[idx])

        for i, (val1, val2) in enumerate(zip(df_etc_latitude, df_etc_longitude)):
            goal = (val1, val2)
            distance = haversine(start, goal, unit = "km")
            if distance <= n:
    #             print(distance)
                if distance < distance_min:
                    distance_min = distance
                    distance_min_idx = i
                    fence_lat[idx] = df_etc_latitude[i]
                    fence_lng[idx] = df_etc_longitude[i]
                    fence_length[idx] = df_etc_length[i]
                    fence_height[idx] = df_etc_height[i]

        # 5km 범위 내에 없는 생태통로
        # 가장 가까운 유도울타리로 매칭
    #     if distance_min == 500:
    #         cnt += 1
    #         for i, (val1, val2) in enumerate(zip(df_temp_latitude, df_temp_longitude)):
    #             goal = (val1, val2)
    #             distance = haversine(start, goal, unit = "km")
    #             if distance < distance_min:
    #                 distance_min = distance
    #                 distance_min_idx = i
        li_distance.append(distance_min)
        li_idx.append(distance_min_idx)
        

#     print(sorted(li_distance, reverse=True))
#     print(cnt)

#         fence_lat[idx] = df_temp_latitude[min_idx]
#         fence_lng[idx] = df_temp_longitude[min_idx]
    if fence_info_return:
        return (fence_lat, fence_lng, fence_length, fence_height)
    return (fence_lat, fence_lng)

In [68]:
fence_lat_1, fence_lng_1 = is_fence(1)
fence_lat_2_3, fence_lng_2_3 = is_fence(2.3)
fence_lat_5, fence_lng_5, fence_length, fence_height = is_fence(5, True)

In [69]:
df_maindata_inner_origin["유도울타리_위도_1km"] = fence_lat_1
df_maindata_inner_origin["유도울타리_경도_1km"] = fence_lng_1
df_maindata_inner_origin["유도울타리_위도_2.3km"] = fence_lat_2_3
df_maindata_inner_origin["유도울타리_경도_2.3km"] = fence_lng_2_3
df_maindata_inner_origin["유도울타리_위도_5km"] = fence_lat_5
df_maindata_inner_origin["유도울타리_경도_5km"] = fence_lng_5
df_maindata_inner_origin["유도울타리_연장_m"] = fence_length
df_maindata_inner_origin["유도울타리_높이_m"] = fence_height

In [70]:
df_maindata_inner_origin.head(10)

,번호,위도,경도,생태통로_유형,차선_수,등산객_이용빈도,생태통로_효율성,이용확인종,이용확인종_수,폭(m),...,식물군락명,등산로까지 최단거리(km),유도울타리_위도_1km,유도울타리_경도_1km,유도울타리_위도_2.3km,유도울타리_경도_2.3km,유도울타리_위도_5km,유도울타리_경도_5km,유도울타리_연장_m,유도울타리_높이_m
0,1495,35.36019,128.06375,0,1,1,3,None,0,NaN,...,소나무군락,3.157026,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.000000
1,1277,37.03408,128.40661,1,0,0,3,None,0,NaN,...,소나무-신갈나무군락,1.087578,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.000000
2,1278,36.89832,128.30563,1,0,0,1,고라니 멧돼지 파충류,3,NaN,...,소나무군락,3.070682,0.0,0.0,0.000000,0.000000,36.927754,128.333896,1028,2.000000
3,1279,36.89687,128.30581,1,0,0,1,고라니 멧돼지 파충류,3,NaN,...,소나무군락,2.921190,0.0,0.0,0.000000,0.000000,36.927754,128.333896,1028,2.000000
4,1090,36.71336,127.42200,0,2,5,1,개 고양이 다람쥐,3,8.23,...,아까시나무-리기다소나무식재림,0.025083,0.0,0.0,36.708802,127.437539,36.708802,127.437539,886,1.571429
5,1023,37.41972,127.18750,0,0,1,2,고라니 너구리 멧돼지 다람쥐,4,25.10,...,신갈나무군락,0.003337,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.000000
6,1527,38.03333,127.19250,0,1,0,3,None,0,NaN,...,리기다소나무식재림,4.207701,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.000000
7,1453,36.40691,129.04714,0,0,0,1,고라니 너구리 멧돼지,3,37.30,...,소나무군락,3.376660,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.000000
8,1515,34.97735,127.81174,0,1,1,2,None,0,30.10,...,곰솔-소나무군락,4.298334,0.0,0.0,34.988906,127.792837,34.988906,127.792837,2050,1.000000
9,1247,37.53230,127.80131,1,0,1,3,None,0,NaN,...,잣나무식재림,3.694382,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.000000


In [71]:
# from collections import Counter

# counter = Counter(li_idx)

# dic = dict(counter)
# for k,v in dic.items():
#     if v > 1:
#         print(k, v)

### Push RDS

In [73]:
table_name = 'MainData'
df_maindata_inner_origin.to_sql(name=table_name, con=db_connection, if_exists='replace', index=False)
# if_exists options : append, fail, replace

497